In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepchem as dc
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

2021-08-26 13:32:43.942609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import random
import tqdm
import copy
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important
# ulimit -n 500000
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['/home/chrisw/Documents/projects/2021/graph-transformer/src'] + sys.path
from dfs_transformer import *

In [5]:
import chemprop as cp
import pandas as pd

In [6]:
config = wandb.config
config.n_atoms = 118
config.n_bonds = 4
config.max_nodes = 400
config.max_edges = 600
config.lr = 0.0003 # 10x smaller than the learning rate of the pretraining seems to be good
config.clip_gradient = 0.5
config.n_epochs = 25
config.patience = 3
config.factor = 0.95
config.minimal_lr = 6e-8
config.target_idx = 7
config.batch_size = 64
config.valid_patience = 5
config.valid_minimal_improvement = 0.00
config.num_workers = 0
config.dataset = 'tox21' # supported 'bbbp', 'clintox', 'hiv', 'tox21'
config.model_dir = '../../models/moleculenet/chemprop_mpnn/%s/'%config.dataset
config.seed = 123
config.data_dir = '../../datasets/mymoleculenet/%s/'%config.dataset

In [7]:
random.seed(config.seed)
torch.manual_seed(config.seed)
np.random.seed(config.seed)

In [8]:
os.makedirs(config.model_dir, exist_ok=True)

In [9]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [10]:
def get_random_scaffold_split(dataset):
    splitter = dc.splits.ScaffoldSplitter()
    shuffled = dataset.complete_shuffle()
    trainidx, valididx, testidx = splitter.split(shuffled)
    train = shuffled.select(trainidx)
    valid = shuffled.select(valididx)
    test = shuffled.select(testidx)
    return train, valid, test

In [11]:
if config.dataset == 'clintox':
    tasks, dsets, transformers = dc.molnet.load_clintox(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'tox21':
    tasks, dsets, transformers = dc.molnet.load_tox21(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'hiv':
    tasks, dsets, transformers = dc.molnet.load_hiv(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'bbbp':
    tasks, dsets, transformers = dc.molnet.load_bbbp(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)

In [12]:
for rep in range(10):
    os.makedirs("%s%d"%(config.data_dir, rep), exist_ok=True)
    trainset, validset, testset = get_random_scaffold_split(dsets[0])
    train_tbl = pd.DataFrame({"smiles":trainset.X, "target":trainset.y[:, -1]})
    valid_tbl = pd.DataFrame({"smiles":validset.X, "target":validset.y[:, -1]})
    test_tbl = pd.DataFrame({"smiles":testset.X, "target":testset.y[:, -1]})
    train_tbl.to_csv("%s%d/train.csv"%(config.data_dir, rep))
    valid_tbl.to_csv("%s%d/valid.csv"%(config.data_dir, rep))
    test_tbl.to_csv("%s%d/test.csv"%(config.data_dir, rep))